# ASASSAS

### Criando objeto do web site

In [1]:
from bs4 import BeautifulSoup
import requests
from typing import List

class Website:
    url: str
    title: str
    content: str
    links: List[str]

    def __init__(self, url):
        self.url = url
        response = requests.get(self.url)
        soup = BeautifulSoup(response.content, "html.parser")
        self.title = soup.title.string if soup.title else "Sem título"

        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.content = soup.body.get_text(separator="\n", strip=True)
        else:
            self.content = ""
        links = [link.get("href") for link in soup.find_all("a")]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title: {self.title}\n\nWebpage contents:{self.content}\n\n"


In [46]:
website_url = "https://anthropic.com"

In [47]:
webobject = Website(website_url)

In [48]:
content_page = webobject.get_contents()

In [50]:
print(content_page)

Webpage Title: Home \ Anthropic

Webpage contents:Skip to main content
Skip to footer
Claude
Chat with Claude
Overview
Max plan
Team plan
Enterprise plan
Explore pricing
Download apps
Claude log in
News
Claude’s character
API
Build with Claude
API overview
Developer docs
Explore pricing
Console log in
News
Learn how to build with Claude
Solutions
Collaborate with Claude
AI agents
Coding
Customer support
Education
Financial services
Case studies
Hear from our customers
Research
Research
Overview
Economic Index
Claude model family
Claude Opus 4.1
Claude Sonnet 4
Claude Haiku 3.5
Research
Claude’s extended thinking
Commitments
Initiatives
Transparency
Responsible scaling policy
Trust center
Security and compliance
Announcement
ISO 42001 certification
Learn
Learning resources
Customer stories
Engineering at Anthropic
Anthropic Academy
Company
About
Careers
Events
Engineering
Building effective agents
News
EN
This is some text inside of a div block.
Try Claude
Try Claude
Try Claude
Download

In [51]:
links = webobject.links
print(links)

['#main', '#footer', 'https://www.anthropic.com/', 'https://www.anthropic.com/claude', 'https://www.anthropic.com/max', 'https://www.anthropic.com/team', 'https://www.anthropic.com/enterprise', 'https://www.anthropic.com/pricing', 'https://claude.ai/download', 'https://claude.ai/', 'https://www.anthropic.com/news/claude-character', 'https://www.anthropic.com/api', 'https://docs.anthropic.com/', 'https://www.anthropic.com/pricing#api', 'https://console.anthropic.com/', 'https://docs.anthropic.com/en/docs/welcome', 'https://www.anthropic.com/solutions/agents', 'https://www.anthropic.com/solutions/coding', 'https://www.anthropic.com/solutions/customer-support', 'https://www.anthropic.com/solutions/education', 'https://www.anthropic.com/solutions/financial-services', 'https://www.anthropic.com/customers', 'https://www.anthropic.com/research', 'https://www.anthropic.com/economic-index', 'https://www.anthropic.com/claude/opus', 'https://www.anthropic.com/claude/sonnet', 'https://www.anthropi

### Setando LLMs

In [52]:
import os
import openai
from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

groq_api_key = os.getenv("GROQ_API_KEY")

client_groq = openai.OpenAI(
    base_url="https://api.groq.com/openai/v1",
    api_key=groq_api_key
)

def get_response_groq(messages):
    response = client_groq.chat.completions.create(
        model="llama3-8b-8192",
        messages=messages,
        response_format={"type": "json_object"}
    )

    return response.choices[0].message.content

### Criando prompts para pegar links mais interessantes

In [54]:
link_system_prompt = """
Você é o melhor em escolher links para a divulgação de uma empresa e na criação de uma folder.[
Você vai receber uma lista de links de uma empresa e deverá escolher os mais interessantes na criação de um folder.
Os links podem ser de páginas da própria empresa ou links irrelevantes.

A saída deverá ser na estrutura de um JSON conforme o exemplo abaixo:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.url/goes/here/careers"}
    ]
}
"""

In [55]:
links_user_prompt = """
Aqui está a lista de links do site: {website_url}

** Lista de links **
{links}
"""

In [56]:
print(links_user_prompt.format(
    website_url=website_url,
    links="\n".join(webobject.links)
))


Aqui está a lista de links do site: https://anthropic.com

** Lista de links **
#main
#footer
https://www.anthropic.com/
https://www.anthropic.com/claude
https://www.anthropic.com/max
https://www.anthropic.com/team
https://www.anthropic.com/enterprise
https://www.anthropic.com/pricing
https://claude.ai/download
https://claude.ai/
https://www.anthropic.com/news/claude-character
https://www.anthropic.com/api
https://docs.anthropic.com/
https://www.anthropic.com/pricing#api
https://console.anthropic.com/
https://docs.anthropic.com/en/docs/welcome
https://www.anthropic.com/solutions/agents
https://www.anthropic.com/solutions/coding
https://www.anthropic.com/solutions/customer-support
https://www.anthropic.com/solutions/education
https://www.anthropic.com/solutions/financial-services
https://www.anthropic.com/customers
https://www.anthropic.com/research
https://www.anthropic.com/economic-index
https://www.anthropic.com/claude/opus
https://www.anthropic.com/claude/sonnet
https://www.anthrop

In [57]:
def get_messages(system_prompt, user_prompt, website_url, links):
    
  user_prompt = user_prompt.format(
    website_url=website_url,
    links="\n".join(links)
  )

  messages=[
    {
      'role': 'system',
      'content': system_prompt,
    },
    {
      'role': 'user',
      'content': user_prompt
    },
  ]

  return messages

### Função que vai retornar os links mais relevantes

In [62]:
messages = get_messages(
    system_prompt=link_system_prompt,
    user_prompt=links_user_prompt,
    website_url=website_url,
    links=webobject.links
)

In [63]:
links_groq = get_response_groq(messages=messages)

In [65]:
import json
links_groq = json.loads(links_groq)

In [66]:
links_groq

{'links': [{'type': 'about page', 'url': 'https://www.anthropic.com'},
  {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'},
  {'type': 'news page', 'url': 'https://www.anthropic.com/news'},
  {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions'},
  {'type': 'customers page', 'url': 'https://www.anthropic.com/customers'},
  {'type': 'research page', 'url': 'https://www.anthropic.com/research'},
  {'type': 'pricings page', 'url': 'https://www.anthropic.com/pricing'},
  {'type': 'api page', 'url': 'https://www.anthropic.com/api'},
  {'type': 'learn page', 'url': 'https://www.anthropic.com/learn'},
  {'type': 'engineering page', 'url': 'https://www.anthropic.com/engineering'},
  {'type': 'company page', 'url': 'https://www.anthropic.com/company'},
  {'type': 'events page', 'url': 'https://www.anthropic.com/events'},
  {'type': 'transparency page',
   'url': 'https://www.anthropic.com/transparency'},
  {'type': 'responsible-scaling-policy page',
   'url

### Fazendo o folder de divulgação

A ideia é entrar nos links e pegar o seu conteúdo

In [73]:
from tqdm import tqdm

def get_info_of_links(links_groq):
    all_links_info = ""
    for link in tqdm(links_groq["links"]):
        all_links_info += f"\n\n{link["type"]}\n"
        print(f"Rodando para {link["url"]}")
        all_links_info += f"{Website(url=link["url"]).get_contents()}"
    return all_links_info

In [74]:
all_links_info = get_info_of_links(links_groq=links_groq)

  0%|          | 0/21 [00:00<?, ?it/s]

Rodando para https://www.anthropic.com


  5%|▍         | 1/21 [00:00<00:07,  2.58it/s]

Rodando para https://www.anthropic.com/careers


 10%|▉         | 2/21 [00:00<00:06,  2.72it/s]

...

 90%|█████████ | 19/21 [00:08<00:00,  3.00it/s]

Rodando para https://console.anthropic.com/
Rodando para https://www.anthropic.com/contact-sales


 95%|█████████▌| 20/21 [00:08<00:00,  3.36it/s]

Rodando para https://www.anthropic.com/jobs


100%|██████████| 21/21 [00:09<00:00,  2.31it/s]


In [76]:
print(all_links_info)



about page
Webpage Title: Home \ Anthropic

Webpage contents:Skip to main content
Skip to footer
Claude
Chat with Claude
Overview
Max plan
Team plan
Enterprise plan
Explore pricing
Download apps
Claude log in
News
Claude’s character
API
Build with Claude
API overview
Developer docs
Explore pricing
Console log in
News
Learn how to build with Claude
Solutions
Collaborate with Claude
AI agents
Coding
Customer support
Education
Financial services
Case studies
Hear from our customers
Research
Research
Overview
Economic Index
Claude model family
Claude Opus 4.1
Claude Sonnet 4
Claude Haiku 3.5
Research
Claude’s extended thinking
Commitments
Initiatives
Transparency
Responsible scaling policy
Trust center
Security and compliance
Announcement
ISO 42001 certification
Learn
Learning resources
Customer stories
Engineering at Anthropic
Anthropic Academy
Company
About
Careers
Events
Engineering
Building effective agents
News
EN
This is some text inside of a div block.
Try Claude
Try Claude
Try Cl

In [78]:
all_info = f"""
Landing page:

{webobject.get_contents()}

{all_links_info}
"""

### Criando prompt para criar o folder

In [80]:
system_prompt = """
Vocè um assistente que analisa o conteúdo de muitas páginas web de uma empresa e cria um folder ou folheto (brochure em inglês), a
respeito da empresa, para clientes prospectivos, investidores e recrutas. Responda em markdown
Inclua detalhes da cultura da empresa, empregos e etc. Tudo que for relevante para este público e claro, se você tiver essa informação
o tragra.
"""

In [82]:
user_prompt = """
Aqui está o conteúdo da empresa: {empresa}

** Conteúdo da empresa **

{conteudo}
"""

### Criando o folheto

In [85]:
def get_messages_final(system_prompt,user_prompt, empresa, conteudo):
    user_prompt = user_prompt.format(
        empresa=empresa,
        conteudo=conteudo
    )

    messages=[
        {
            'role': 'system',
            'content': system_prompt,
        },
        {
            'role': 'user',
            'content': user_prompt[:5000]
        },
    ]

    return messages

In [86]:
messages_final = get_messages_final(system_prompt=system_prompt, user_prompt=user_prompt, empresa="Anthropic", conteudo=all_info)

In [87]:
def create_folder(messages):
    response = client_groq.chat.completions.create(
        model="llama3-8b-8192",
        messages=messages
    )

    return response.choices[0].message.content

In [88]:
final_response = create_folder(messages=messages_final)

In [89]:
from IPython.display import display, Markdown

display(Markdown(final_response))

**Anthropic Brochure**

**Introduction**

Welcome to Anthropic, a company that is revolutionizing the world of Artificial Intelligence (AI) by putting safety at the forefront of our Research and Development. Our mission is to build AI that serves humanity's long-term well-being.

**Our Story**

Anthropic was founded on the idea that AI has the potential to be a powerful force for good, but only if it is designed and developed responsibly. Our team of experts in AI research and development is committed to building tools that benefit humanity, like Claude, our most intelligent AI model.

**Our Values**

At Anthropic, we believe that:

* Knowledge is power, but it's up to us to ensure that power is used responsibly.
* Designing powerful technologies requires both bold steps forward and intentional pauses to consider the effects.
* Building tools with human benefit at their foundation is essential for a better future.

**Our Approach**

We take a holistic approach to AI development, focusing on building tools that are not only powerful but also safe and responsible. Our Research and Development team is committed to:

* Developing AI models that are designed with safety and transparency in mind.
* Collaborating with experts in various fields to ensure that our AI tools are aligning with human values.
* Creating educational resources and training programs to help others build with Claude.

**Our Products**

Our flagship product is Claude, a powerful AI model that can handle hours of work across Claude and Claude Code. Claude is available on three different plans: Max, Team, and Enterprise.

* **Max Plan**: Ideal for individuals and small teams, this plan provides access to Claude's core features.
* **Team Plan**: Designed for medium-sized teams, this plan offers additional features and support.
* **Enterprise Plan**: For large enterprises, this plan provides customizable solutions and dedicated support.

**Our Research**

We are committed to driving innovation in AI research and development. Our research focus areas include:

* **AI Safety**: We are dedicated to ensuring that our AI tools are safe and responsible.
* **Societal Impacts**: We believe that AI should be designed to benefit humanity and society.
* **Interpretability**: We are working to create AI models that are transparent and interpretable.

**Join Our Team**

If you're passionate about building a safer and more responsible future for AI, we'd love to hear from you. We have a range of open roles available, from Research Scientists to Software Engineers.

**Contact Us**

To learn more about Anthropic and our products, please visit our website or contact us at [info@anthropic.com](mailto:info@anthropic.com).

**Stay Connected**

Follow us on social media to stay up-to-date on our latest news, research, and product updates.

[LinkedIn](https://www.linkedin.com/company/anthropic/) | [Twitter](https://twitter.com/anthropic)